In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras 

In [3]:
imdb = keras.datasets.imdb

In [4]:
(train_data , train_label) , (test_data , test_label) = imdb.load_data()

In [5]:
word_index = imdb.get_word_index()

In [6]:
len(word_index)

88584

In [7]:
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2 
word_index["<UNUSED>"] = 3

In [8]:
reverse_word_dic = dict([(value,key) for (key,value) in word_index.items()])

In [9]:
def decode_text(text):
    return ' '.join([reverse_word_dic.get(i,'?') for i in text])

In [10]:
decode_text(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh

In [11]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [12]:

vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.leaky_relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss = 'binary_crossentropy',
              metrics= ['accuracy'])

In [14]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_label[:10000]
partial_y_train = train_label[10000:]

In [15]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=60,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/60
15000/15000 [==============================] - ETA: 0s - loss: 0.6917 - acc: 0.521 - 3s 211us/step - loss: 0.6916 - acc: 0.5228 - val_loss: 0.6894 - val_acc: 0.5658
Epoch 2/60
15000/15000 [==============================] - 1s 38us/step - loss: 0.6845 - acc: 0.6859 - val_loss: 0.6793 - val_acc: 0.7432
Epoch 3/60
15000/15000 [==============================] - 1s 44us/step - loss: 0.6683 - acc: 0.7679 - val_loss: 0.6584 - val_acc: 0.7632
Epoch 4/60
15000/15000 [==============================] - 1s 42us/step - loss: 0.6387 - acc: 0.7825 - val_loss: 0.6257 - val_acc: 0.7727
Epoch 5/60
15000/15000 [==============================] - 1s 43us/step - loss: 0.5962 - acc: 0.8067 - val_loss: 0.5816 - val_acc: 0.7956
Epoch 6/60
15000/15000 [==============================] - 1s 42us/step - loss: 0.5457 - acc: 0.8227 - val_loss: 0.5347 - val_acc: 0.8124
Epoch 7/60
15000/15000 [==============================] - 1s 42us/step - loss: 0.4933 - a

15000/15000 [==============================] - 1s 39us/step - loss: 0.0386 - acc: 0.9939 - val_loss: 0.4133 - val_acc: 0.8718
Epoch 58/60
15000/15000 [==============================] - 1s 38us/step - loss: 0.0370 - acc: 0.9942 - val_loss: 0.4174 - val_acc: 0.8744
Epoch 59/60
15000/15000 [==============================] - 1s 38us/step - loss: 0.0353 - acc: 0.9945 - val_loss: 0.4239 - val_acc: 0.8721
Epoch 60/60
15000/15000 [==============================] - 1s 39us/step - loss: 0.0338 - acc: 0.9951 - val_loss: 0.4311 - val_acc: 0.8709


In [16]:
predictions = model.predict(test_data)

In [17]:
results = model.evaluate(test_data,test_label)

25000/25000 [==============================] - 1s 47us/step


In [18]:
results

[0.46178459196567534, 0.85856]